In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, TrainerCallback
from trl import AutoModelForCausalLMWithValueHead, DPOTrainer, DPOConfig
from huggingface_hub import login

from openai import OpenAI
from difflib import SequenceMatcher

from datasets import Dataset
import pandas as pd

import os
from pathlib import Path
from dotenv import load_dotenv

import DPO_pairwise_human as dp

load_dotenv()
PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT")).resolve()
MODEL_ROOT = Path(os.getenv("MODEL_ROOT")).resolve()
DATA_ROOT = Path(os.getenv("DATA_ROOT")).expanduser().resolve()

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
# show the GPU device and number of GPUs available
print(f"Using device: {device}")
print(f"Number of GPUs available: {torch.cuda.device_count()}")

Using device: cuda
Number of GPUs available: 1


In [12]:
print("loading model and tokenizer...")
model_path = MODEL_ROOT / 'test' / 'TinyLlama-1.1B-Chat-v1.0'

tokenizer = AutoTokenizer.from_pretrained(str(model_path))
model = AutoModelForCausalLMWithValueHead.from_pretrained(str(model_path)).to(device)
model.warnings_issued = {}
print("model loaded")

loading model and tokenizer...


Some weights of the model checkpoint at models/test/TinyLlama-1.1B-Chat-v1.0 were not used when initializing LlamaForCausalLM: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing LlamaForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LlamaForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model loaded


In [13]:
print('loding evaluation dataset...')
dataset_path = DATA_ROOT / '.kaggle' / 'cnn_dailymail' / 'validation.csv'
df = pd.read_csv(dataset_path, nrows=1)
df = df[['article', 'highlights']].rename(columns={
    'article': 'content',
    'highlights': 'answer'
})
dataset = Dataset.from_pandas(df)
print('dataset loaded')

loding evaluation dataset...
dataset loaded


In [14]:
raw_data = dp.prepare_raw_data(model, tokenizer, dataset)

pairwise_data = dp.prepare_pairwise_data(raw_data)

Preparing raw data...


Processed: 1/1
Raw data preparation complete.
Preparing pairwise compared data...
Processed: 1/1
Pairwise data preparation complete.


In [15]:
score = dp.compute_win_rate(model, tokenizer, pairwise_data)

In [9]:
print(score)

1.0


In [16]:
print(raw_data[0])
print(pairwise_data[0])

{'prompt': "Sally Forrest, an actress-dancer who graced the silver screen throughout the '40s and '50s in MGM musicals and films such as the 1956 noir While the City Sleeps died on March 15 at her home in Beverly Hills, California. Forrest, whose birth name was Katherine Feeney, was 86 and had long battled cancer. Her publicist, Judith Goffin, announced the news Thursday. Scroll down for video . Actress: Sally Forrest was in the 1951 Ida Lupino-directed film 'Hard, Fast and Beautiful' (left) and the 1956 Fritz Lang movie 'While the City Sleeps' A San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films including the critical and commercial success Not Wanted, Never Fear and Hard, Fast and Beautiful. Some of Forrest's other film credits included Bannerline, Son of Sinbad, and Excuse My Dust, according to her iMDB\xa0page. The page also indicates Forrest was in multiple Climax! and Rawhide television episodes. Forrest appeare